# ✅ Imports

In [10]:
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import load_model
import numpy as np
import os


In [11]:
os.listdir('/kaggle/input') 

['cifake-real-and-ai-generated-synthetic-images']

# ✅ Model Builder

In [12]:
def build_vgg19():
    base = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base.layers:
        layer.trainable = False
    x = Flatten()(base.output)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    out = Dense(1, activation='sigmoid')(x)
    return Model(base.input, out)

# ✅ Data Generators

In [13]:
import shutil
from sklearn.model_selection import train_test_split

base_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train"
val_dir = "/kaggle/working/validation"

for label in ['REAL', 'FAKE']:
    os.makedirs(os.path.join(val_dir, label), exist_ok=True)
    files = os.listdir(os.path.join(base_dir, label))
    train_files, val_files = train_test_split(files, test_size=0.2, random_state=42)

    for fname in val_files:
        src = os.path.join(base_dir, label, fname)
        dst = os.path.join(val_dir, label, fname)
        shutil.copyfile(src, dst)


In [14]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train',
    target_size=(224, 224), batch_size=32, class_mode='binary', shuffle=True)

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    '/kaggle/working/validation',
    target_size=(224, 224), batch_size=32, class_mode='binary', shuffle=False)

test_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test',
    target_size=(224, 224), batch_size=32, class_mode='binary', shuffle=False)

Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


# ✅ FGSM Attack Function and PGD Attack Function

In [15]:
@tf.function
def fgsm_attack(model, images, labels, epsilon=0.05):
    with tf.GradientTape() as tape:
        tape.watch(images)
        predictions = model(images)
        loss = tf.keras.losses.BinaryCrossentropy()(labels, predictions)
    gradient = tape.gradient(loss, images)
    signed_grad = tf.sign(gradient)
    adv_images = images + epsilon * signed_grad
    return tf.clip_by_value(adv_images, 0, 1)

# ✅ PGD Attack Function
@tf.function
def pgd_attack(x, y, model, loss_fn, eps=4/255, alpha=1/255, iters=7):
    x_adv = tf.identity(x)
    for _ in tf.range(iters):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            pred = model(x_adv, training=False)
            loss = loss_fn(y, pred)
        grad = tape.gradient(loss, x_adv)
        x_adv = x_adv + alpha * tf.sign(grad)
        x_adv = tf.clip_by_value(x_adv, x - eps, x + eps)
        x_adv = tf.clip_by_value(x_adv, -1.0, 1.0)
    return x_adv

# ✅ Train and Save Baseline Model

In [16]:
# Paths to save model and weights
weights_path_baseline = '/kaggle/working/vgg19_baseline_cifake.weights.h5'
model_path_baseline   = '/kaggle/working/vgg19_baseline_cifake_model.h5'
if os.path.exists(model_path_baseline):
    print("Loading saved model...")
    baseline_model = load_model(model_path_baseline)
else:
    print("Training baseline model...")
    baseline_model = build_vgg19()
    baseline_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    baseline_model.fit(train_gen, epochs=3, validation_data=val_gen)
    baseline_model.save_weights(weights_path_baseline)
    baseline_model.save(model_path_baseline)
    print("Baseline model trained and saved.")

print("\nEvaluating Baseline Model on Clean Test Set:")
baseline_clean_acc = baseline_model.evaluate(test_gen)
print("Baseline Clean Accuracy:", baseline_clean_acc)


Training baseline model...


I0000 00:00:1744448678.497597      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1744448678.498416      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/3


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1744448686.752826     122 service.cc:148] XLA service 0x7c029800fc60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744448686.758094     122 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1744448686.758117     122 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1744448687.342868     122 cuda_dnn.cc:529] Loaded cuDNN version 90300


   1/3125 ━━━━━━━━━━━━━━━━━━━━ 15:06:27 17s/step - accuracy: 0.5938 - loss: 2.9820

I0000 00:00:1744448699.862714     122 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3125/3125 ━━━━━━━━━━━━━━━━━━━━ 849s 266ms/step - accuracy: 0.8535 - loss: 0.6168 - val_accuracy: 0.9240 - val_loss: 0.1842
Epoch 2/3
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 825s 264ms/step - accuracy: 0.9005 - loss: 0.2454 - val_accuracy: 0.9391 - val_loss: 0.1559
Epoch 3/3
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 826s 264ms/step - accuracy: 0.9122 - loss: 0.2194 - val_accuracy: 0.9445 - val_loss: 0.1361
Baseline model trained and saved.

Evaluating Baseline Model on Clean Test Set:
625/625 ━━━━━━━━━━━━━━━━━━━━ 153s 245ms/step - accuracy: 0.9280 - loss: 0.1888
Baseline Clean Accuracy: [0.18823879957199097, 0.9305499792098999]


# ✅ FGSM Vulnerability Evaluation

In [17]:
x_test_sample, y_test_sample = next(test_gen)
x_test_tensor = tf.convert_to_tensor(x_test_sample, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test_sample, dtype=tf.float32)

x_fgsm_test = fgsm_attack(baseline_model, x_test_tensor, y_test_tensor)
print("\nEvaluating Baseline Model on FGSM-Adversarial Examples:")
baseline_fgsm_acc = baseline_model.evaluate(x_fgsm_test, y_test_sample)
print("Baseline Accuracy on FGSM-Adversarial:", baseline_fgsm_acc)


Evaluating Baseline Model on FGSM-Adversarial Examples:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step - accuracy: 0.8438 - loss: 0.3812
Baseline Accuracy on FGSM-Adversarial: [0.3812202513217926, 0.84375]


# ✅ Adversarial Training with FGSM + PGD

In [ ]:
# Paths
adv_weights_path = '/kaggle/working/vgg19_adv_trained.weights.h5'
adv_model_path   = '/kaggle/working/vgg19_adv_trained_model.h5'

# Load if already saved
if os.path.exists(adv_model_path):
    print("Loading saved adversarially trained model...")
    adv_model = load_model(adv_model_path)
else:
    print("Training adversarially robust model...")
    adv_model = build_vgg19()
    optimizer = tf.keras.optimizers.Adam(1e-4)
    loss_fn = tf.keras.losses.BinaryCrossentropy()

    epochs = 3
    train_steps = train_gen.samples // train_gen.batch_size
    val_steps = val_gen.samples // val_gen.batch_size

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        for step in range(train_steps):
            x_batch, y_batch = next(train_gen)
            x_tensor = tf.convert_to_tensor(x_batch, dtype=tf.float32)
            y_tensor = tf.convert_to_tensor(y_batch, dtype=tf.float32)

            adv_count = x_tensor.shape[0] // 2
            x_fgsm = fgsm_attack(adv_model, x_tensor[:adv_count], y_tensor[:adv_count])
            x_pgd = pgd_attack(x_tensor[adv_count:], y_tensor[adv_count:], adv_model, loss_fn)

            x_mix = tf.concat([x_fgsm, x_pgd], axis=0)
            y_mix = y_tensor

            with tf.GradientTape() as tape:
                preds = adv_model(x_mix, training=True)
                loss = loss_fn(y_mix, preds)
            grads = tape.gradient(loss, adv_model.trainable_variables)
            optimizer.apply_gradients(zip(grads, adv_model.trainable_variables))

        val_acc = tf.keras.metrics.BinaryAccuracy()
        for _ in range(val_steps):
            xv, yv = next(val_gen)
            val_acc.update_state(yv, adv_model(xv, training=False))
        print("Validation Accuracy:", val_acc.result().numpy())

    print("\nTraining done. Saving adversarially trained model...")
    adv_model.save_weights(adv_weights_path)
    adv_model.save(adv_model_path)

# Evaluation (done in both cases)
print("Evaluating Adv-Trained Model on Clean Test Set:")
adv_clean_acc = adv_model.evaluate(test_gen)
print("Adversarially Trained Accuracy on Clean Test:", adv_clean_acc)

x_adv_fgsm_eval = fgsm_attack(adv_model, x_test_tensor, y_test_tensor)
print("Evaluating Adv-Trained Model on FGSM-Adversarial Examples:")
adv_fgsm_acc = adv_model.evaluate(x_adv_fgsm_eval, y_test_sample)
print("Adv-Trained Accuracy on FGSM-Adversarial:", adv_fgsm_acc)


Training adversarially robust model...

Epoch 1/3


E0000 00:00:1744451352.022107      31 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: MutableGraphView::SortTopologically error: detected edge(s) creating cycle(s) {'while/body/_1/while/Identity_3' -> 'while/next_iteration/_76'}.


# ✅ Evaluate and Save Adversarially Trained Model

In [ ]:
print("Evaluating Adv-Trained Model on Clean Test Set:")
adv_clean_acc = adv_model.evaluate(test_gen)
print("Adversarially Trained Accuracy on Clean Test:", adv_clean_acc)
    
x_adv_fgsm_eval = fgsm_attack(adv_model, x_test_tensor, y_test_tensor)
print("Evaluating Adv-Trained Model on FGSM-Adversarial Examples:")
adv_fgsm_acc = adv_model.evaluate(x_adv_fgsm_eval, y_test_sample)
print("Adv-Trained Accuracy on FGSM-Adversarial:", adv_fgsm_acc)
    
# ✅ Save adversarially trained model
adv_model.save_weights('/kaggle/working/vgg19_adv_trained.weights.h5')
adv_model.save('/kaggle/working/vgg19_adv_trained_model.h5')
print("Adversarially trained model saved.")
print("Adv-Trained Accuracy on FGSM-Adversarial:", adv_fgsm_acc)